In [81]:
#!/usr/bin/python3

from login_details import engine1
from pandas import DataFrame
import pandas as pd

In [82]:
# get data from stoppage_id db
with engine1.connect() as con:
    stoppage_hist_query = con.execute("""
    SELECT id, stoppage_id, datetime
    FROM dg_ln_stoppages_raw
    ORDER BY datetime ASC
    LIMIT 100
    """)
    con.close()

In [83]:
# get data from stoppage_string db
with engine1.connect() as conn:
    stoppage_str_query = conn.execute("""
    SELECT fault_id, fault_str, line_stopped, primary_reason
    FROM dg_stoppage_strings
    ORDER BY fault_id ASC
    """)
    con.close()

In [84]:
# convert database instances to DataFrame

# converting stoppage string instance
stoppage_str_df = DataFrame(stoppage_str_query.fetchall())
stoppage_str_df.columns = stoppage_str_query.keys()
stoppage_str_df.set_index('fault_id')

# converting stoppage history instance
stoppage_history_df = DataFrame(stoppage_hist_query.fetchall())
stoppage_history_df.columns = stoppage_hist_query.keys()


In [85]:
# create the "occurred_at" column with the correct datetime format
stoppage_history_df['occurred_at'] = pd.to_datetime(stoppage_history_df['datetime'].dt.strftime('%d-%m-%Y %H:%M:%S'))

# remove original datetime
del stoppage_history_df['datetime']
del stoppage_history_df['id']

stoppage_history_df.sort_values(by='occurred_at')

,stoppage_id,occurred_at
0,0,2021-10-19 06:49:43
1,64,2021-10-19 06:55:17
2,49,2021-10-19 06:55:19
3,1,2021-10-19 06:55:51
4,59,2021-10-19 06:55:53
...,...,...
95,59,2021-10-19 10:31:58
96,0,2021-10-19 10:32:18
97,42,2021-10-19 10:42:08
98,0,2021-10-19 10:43:56


In [113]:
var = stoppage_str_df.loc[5, 'fault_id']
#stoppage_str_df_2['fault_id'] == 0
print(var['fault_str'])

IndexError: invalid index to scalar variable.

In [108]:
#from numpy.compat import basestring

# Iterate through the dataframe

for index_history, stoppage_id in stoppage_history_df.iterrows():
    #stoppage_str_df_2 = stoppage_str_df.loc[stoppage_str_df['fault_id'] == int(stoppage_id[0])]
    print(stoppage_str_df.loc[int(stoppage_id[0]), 'fault_id'])
    #print(stoppage_str_df_2)
    #stoppage_history_df['stoppage_reason'] = stoppage_str_df_2['fault_str']
    #stoppage_history_df['stoppage_reason']= stoppage_str_df_2['fault_str']
    #print(stoppage_history_df['stoppage_reason'])

TypeError: int() argument must be a string, a bytes-like object or a number, not 'Timestamp'

In [63]:
print(stoppage_history_df)

    stoppage_id         occurred_at stoppage_reason
0             0 2021-10-19 06:49:43             NaN
1            64 2021-10-19 06:55:17             NaN
2            49 2021-10-19 06:55:19             NaN
3             1 2021-10-19 06:55:51             NaN
4            59 2021-10-19 06:55:53             NaN
..          ...                 ...             ...
95           59 2021-10-19 10:31:58             NaN
96            0 2021-10-19 10:32:18             NaN
97           42 2021-10-19 10:42:08             NaN
98            0 2021-10-19 10:43:56             NaN
99           44 2021-10-19 10:44:28             NaN

[100 rows x 3 columns]


In [120]:
# print rows based on stoppage_id value
# stoppage_history_df.loc[stoppage_history_df['stoppage_id']==0]

,stoppage_id,occurred_at
0,0,2021-10-19 06:49:43
5,0,2021-10-19 06:55:57
7,0,2021-10-19 07:00:12
10,0,2021-10-19 07:14:04
14,0,2021-10-19 07:15:52
16,0,2021-10-19 07:18:20
18,0,2021-10-19 07:23:56
20,0,2021-10-19 07:30:14
25,0,2021-10-19 07:34:14
31,0,2021-10-19 08:02:46
